In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from definitions import *
from scipy import integrate

# $Q$-range corrections

## Their definitions
Rewriting to normal Euclidean coordinates,
$$G_1(\delta) = \frac{1}{2\pi\xi_t}\int_0^\infty J_0(Q\delta)\dfrac{d\sigma(Q)}{d\Omega}QdQ \equiv \frac{1}{4\pi^2\xi_t}\int_{-\infty}^\infty\int_{-\infty}^\infty J_0(Q\delta)\dfrac{d\sigma(Q)}{d\Omega}dQ_xdQ_y$$
$$s(\delta) = G_1(\delta)\xi_t 4\pi^2 t$$
$$h(\delta) = Tk_0^2[e^{s(\delta)/k_0^2}-1]$$
$$T = e^{-\sigma t}$$
## My definitions
$$G_2(\delta) = \frac{t}{k_0^2}\int_{-\infty}^\infty\int_{-\infty}^\infty \cos(Q\delta)\dfrac{d\sigma(Q)}{d\Omega}dQ_xdQ_y$$
## Mapping
$$G_1(\delta) = \frac{1}{\xi_t t \lambda_0^2}G_2(\delta)$$

So
$$s(\delta) = G_2(\delta)k_0^2$$
$$h(\delta) = Tk_0^2[e^{s(\delta)/k_0^2}-1] = Tk_0^2[e^{G_2(\delta)}-1]$$
Finally, using
$$H(Q) = \frac{1}{2\pi}\int_0^\infty J_0(Q\delta)h(\delta)\delta d\delta$$


$$V_s(\delta) = \frac{T + \frac{2\pi}{k_0^2}\int_0^{Q_{max}} J_0(Q\delta)H(Q)Q dQ}{T + \frac{2\pi}{k_0^2}\int_0^{Q_{max}} H(Q)Q dQ}$$

The bottom expression is a trick: instead of cancelling the modulation by adding measurements from the two analyser settings, ($I_{sum}$), you construct this by considering the 'modulation' at $\delta=0$. In theory, this is the same as everything is polarized in the same way. 

This can be verified by seeing that they simplify when $Q_{max} \to \infty$ to
$$V_s(\delta) = \frac{T + h(\delta)/k_0^2}{T + h(0)/k_0^2} = e^{[s(\delta) - s(0)]/k_0^2}$$

## Final expression
The question is: can I now do a slightly forbidden thing by using the variant with cosine transform
$$H(Q_y) = \frac{1}{2\pi}\int_0^\infty \cos(Q_y\delta)h(\delta) d\delta$$
and
$$V_s(\delta) = \frac{T + \frac{2\pi}{k_0^2}\int_{-Q_{max}}^{Q_{max}}\int_{-Q_{max}}^{Q_{max}} \cos(Q_y\delta)H(Q_y) dQ_xdQ_y}{T + \frac{2\pi}{k_0^2}\int_{-Q_{max}}^{Q_{max}}\int_{-Q_{max}}^{Q_{max}} H(Q_y) dQ_xdQ_y}$$

All constants before integrals are uncertain...

### Between step
I'm not entirely sure whether the above correction will work. After all, the integral is not a function of Q_x anymore, meaning that this will diverge. In 1D, a starting might be to ignore the effect of the $x$-integration, assuming an infinitely wide detector that picks up all $Q$ along that axis. Even then, there will be a correction due to the incomplete cosine transformation.
$$V_s(\delta) = \frac{T + \frac{2\pi}{k_0^2}\int_{-Q_{max}}^{Q_{max}} \cos(Q_y\delta)H(Q_y) dQ_y}{T + \frac{2\pi}{k_0^2}\int_{-Q_{max}}^{Q_{max}} H(Q_y) dQ_y}$$
Assuming isotropic scattering so that $\dfrac{d\sigma}{d\Omega}(Q_x, Q_y) = \dfrac{d\sigma}{d\Omega}\left(\sqrt{Q_x^2 + Q_y^2}\right) = \dfrac{d\sigma}{d\Omega}(Q)$,


# From $G(\delta)$ to $I(Q)$ using Hankel transforms

In [ ]:
R = 1000e-10 # m
delta = np.linspace(0,4e-7,1000000)
d_delta = delta[1] - delta[0]
xi = delta / R
G_xi = G(delta, R)
# G_xi_2 = G(delta, 2 * R)

plt.plot(delta*1e9,G_xi)
# plt.plot(delta*1e9,G_xi_2)
plt.xlabel(r'$\delta$ [nm]')
plt.ylabel(r'$G(\delta)$')
plt.grid()

# $\sigma_t$ correction factor 

Assuming a Gaussian $\lambda$ distribution, a more correct expression for $\sigma_t$ averaged over all wavelengths can be derived from the second moment of the distribution. $\tau \propto \lambda^2$, meaning that $E[\lambda^2]$ gives the right value. For a normal distribution $N(\lambda_0, \sigma^2)$ it can be derived that as $Var(\lambda) = \sigma = \sqrt{E[\lambda^2] - E[\lambda]^2} = \sqrt{E[\lambda^2] - \lambda_0^2}$, 
$$E[\lambda^2] = \lambda_0^2 + \sigma^2$$
This means that the true average $\bar{\tau}$ over all $\lambda$ is related to $\tau(\lambda_0)$ by
$$\frac{\bar{\tau}}{\tau(\lambda_0)} = 1 + \frac{\sigma^2}{\lambda_0^2}$$

In the most extreme case, $\Delta\lambda/\lambda_0 = 0.1$ which gives $\sigma/\lambda_0 = 0.042467$. This gives a correction factor of $1.001803$ or only about $0.2\%$. 



In [ ]:
0.04246609001440096**2

In [ ]:
from scipy.special import j0
fig, ax = plt.subplots()
x = np.linspace(0, 0.2, 1000)
for f in [1,5,10,30,100,250,1000]:
    y = j0(x * f)
    ax.plot(x, y)
plt.show()

In [ ]:
N_Q = 1000
Q_min = 0.1/R
Q_max = 100/R
Q_range = np.linspace(Q_min, Q_max,N_Q)
# print(Q_range)
# Printed in Å^-1
Q_min*1e-10, Q_max*1e-10

In [ ]:
from scipy import fft
import numpy as np
import matplotlib.pyplot as plt

mu = 0.0 # Order of Bessel function
r = np.logspace(-7, 1, 128)  # Input evaluation points
dln = np.log(r[1]/r[0])      # Step size
offset = fft.fhtoffset(dln, initial=-6.*np.log(10), mu=mu, bias=-1.5)
k = np.exp(offset)/r[::-1]   # Output evaluation points

a = 0.5
f = np.exp(-a*r)             # This is what I want to transform
fk = k/np.sqrt((k**2+a**2)**3) # This is what I expect analytically (I looked it up from a table)
# f_fht = fft.fht(f*r, dln, offset=offset, mu=mu)/k
f_fht = fft.fht(f*r, dln, offset=offset, mu=mu, bias=-1.5)/k
plt.loglog(k,fk,k,f_fht,'--')
plt.show()

# My attempt at numerical Hankel transform (it didn't go great...)

In [ ]:
def hankel_transform(Q):
    # delta = np.linspace(0,7e-7,1000)
    # xi = delta / R
    # G_xi = G(delta, R)
    C = 1e-5
    # print(Q.shape)
    integrands = j0(Q *delta) * G_xi * Q *C
    nan_ix = np.isnan(integrands)
    integrands[nan_ix] = 0

    # print(integrands)
    # plt.plot(delta,integrands,'.')
    # plt.show()
    I_Q = integrate.trapezoid(integrands, dx = d_delta)
    return 2 * np.pi * I_Q / C
I = np.zeros_like(Q_range)
for i in range(N_Q):
    # print(Q_range[i])
    I[i] = hankel_transform(Q_range[i])


# Example numerical Hankel transform
The Hankel transform of function
$$f(r) = e^{-ar}$$
is
$$F(k) = \frac{1}{\sqrt{k^2 + a^2}^3}$$

In [ ]:
from scipy import fft
import numpy as np
import matplotlib.pyplot as plt

mu = 1.0 # Order of Bessel function
r = np.logspace(-7, 1, 128)  # Input evaluation points
dln = np.log(r[1]/r[0])      # Step size
# offset = fft.fhtoffset(dln, initial=-6.*np.log(10), mu=mu)
offset = fft.fhtoffset(dln, initial=-6.*np.log(10), mu=mu, bias=-1.5)
k = np.exp(offset)/r[::-1]   # Output evaluation points

a = 0.5
f = np.exp(-a*r)             # This is what I want to transform
fk = k/np.sqrt((k**2+a**2)**3) # This is what I expect analytically (I looked it up from a table)
# f_fht = fft.fht(f*r, dln, offset=offset, mu=mu)/k
f_fht = fft.fht(f*r, dln, offset=offset, mu=mu, bias=-1.5)/k
plt.loglog(k,fk,k,f_fht,'--')
plt.show()

In [ ]:
Q_range, I
plt.plot(Q_range,I)
plt.yscale('log')